# Lab 3: Genome-Wide Association Studies

### Due March 25, 2020 11:59 PM

#### Preamble (Don't change this)

## Important Instructions - 

1. Please implement all the *graded functions* in main.py file. Do not change function names in main.py.
2. Please read the description of every graded function very carefully. The description clearly states what is the expectation of each graded function. 
3. After some graded functions, there is a cell which you can run and see if the expected output matches the output you are getting. 
4. The expected output provided is just a way for you to assess the correctness of your code. The code will be tested on several other cases as well.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [180]:
%run main.py

In [181]:
module = Lab3()

## GWAS from dogs genotype data

In this lab, you will perform GWAS on genotype data from dogs.
The binary phenotype considered will be "yellow fur color".
Hence, we will be trying to identify SNPs in a dog genome that are associated with yellow fur.

You will work with the file dog_genotype.vcf.
The first few lines of the file start with ``##`` and can be ignored, as they contain general information about the file.
The header line starts with ``#`` and contains the names of each of the columns in the matrix.
All the remaining lines correspond to the data about a specific SNP.

Notice that the columns are separated by tab ("\t").

In [1]:
initial_lines=[]
header_line=""
snp_lines=[]

for cur_line in open("dog_genotype.vcf","r"):
    if cur_line[:2] == "##":
        initial_lines.append(cur_line)
    elif cur_line[:1] == "#":
        header_line = cur_line
    else:
        snp_lines.append(cur_line)

In [16]:
print(header_line)
index = []
flag = 0
str1 = ''
for i in range(len(header_line)):
    if (header_line[i] == '\t' or header_line[i] == '\n') and len(str1) != 0:
        index.append(str1)
        flag = 0
        str1 = ''
    if flag == 1:
        str1 = str1 + header_line[i]
    if flag == 0 and header_line[i] == 'd':
        str1 = str1 + 'd'
        flag = 1
    elif flag == 0 and header_line[i] == 'y':
        str1 = str1 + 'y'
        flag = 1

print(index)

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	dark_13	dark_23	dark_21	yellow_5	yellow_6	dark_1	dark_7	dark_9	dark_2	dark_10	dark_8	yellow_24	yellow_22	dark_17	dark_18	dark_4	dark_6	dark_5	yellow_16	yellow_14	yellow_18	yellow_15	yellow_13	yellow_17	yellow_12	yellow_10	yellow_11	dark_11	dark_19	dark_20	dark_12	dark_3	dark_28	dark_25	dark_26	dark_29	dark_27	yellow_8	yellow_1	yellow_3	dark_14	dark_15	yellow_7	yellow_2	dark_22	yellow_4	yellow_9	yellow_31	dark_24	dark_31	yellow_23	yellow_20	yellow_21

['dark_13', 'dark_23', 'dark_21', 'yellow_5', 'yellow_6', 'dark_1', 'dark_7', 'dark_9', 'dark_2', 'dark_10', 'dark_8', 'yellow_24', 'yellow_22', 'dark_17', 'dark_18', 'dark_4', 'dark_6', 'dark_5', 'yellow_16', 'yellow_14', 'yellow_18', 'yellow_15', 'yellow_13', 'yellow_17', 'yellow_12', 'yellow_10', 'yellow_11', 'dark_11', 'dark_19', 'dark_20', 'dark_12', 'dark_3', 'dark_28', 'dark_25', 'dark_26', 'dark_29', 'dark_27', 'yellow_8', 'yellow_1', 'yellow_3', 'dark_14', 'dark_15', 'yellow_7', 'yello

As you can see above, the header line contains SNP information such as chromosome, position,  SNP id, REF, and ALT.

The columns dark_13 until yellow_21 represent 53 dogs. Their ids encode whether they have yellow or dark fur (coat).

In [40]:
for line in snp_lines[-5:]:
    print(line)

39	118625882	rs24656710,BICF2P1205853	T	C	.	.	PR	GT	0/0	0/1	0/1	0/0	0/1	1/1	0/1	0/0	0/1	./.	1/1	0/0	0/0	0/0	0/0	0/0	0/0	0/0	./.	0/0	0/1	0/1	1/1	0/0	0/0	1/1	0/0	0/0	0/0	0/0	0/1	0/0	0/1	1/1	0/0	0/1	1/1	0/0	0/1	0/0	0/0	0/1	0/1	0/0	0/0	0/0	1/1	0/0	0/0	0/0	1/1	0/0	0/1

39	121038139	BICF2G6302258	T	C	.	.	PR	GT	0/0	0/1	0/1	0/1	0/0	0/1	0/1	0/1	0/1	0/1	0/1	0/0	0/0	0/1	0/0	0/1	0/1	0/1	0/1	0/1	0/1	0/1	0/1	0/0	0/1	0/0	0/1	0/1	0/1	0/1	0/1	0/1	0/1	0/0	0/1	0/1	0/1	0/1	0/1	0/0	0/1	0/0	0/1	0/1	0/1	0/1	0/1	0/1	0/1	0/1	0/1	0/1	0/1

39	122458103	BICF2G6302096	T	C	.	.	PR	GT	0/0	0/1	0/1	0/0	0/0	./.	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/1	0/0	1/1	1/1	0/0	1/1	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0

39	122478098	chrX:122478098	T	C	.	.	PR	GT	0/0	0/0	./.	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/0	0/1	0/0	0/0	0/0	0/1	0/0	0/0	0/0	0/0	./.	0/0	0/0	0/0	0/0	0/0	0/0	0/1	0/0	0/1	0/0	0/0	0/0	0/0	0/0	0/1	0/0	0/1	0/1	0/0	0/0	0/1	0/0	0

Notice that the genotypes are encoded as x/y, where x and y are in {0,1,.}. The '.' represents a genotype that is not known.

## Graded Function 1 : create_data() (10 marks)

Purpose - To prepare data for association analysis. We will prepare a dataframe which has 53 rows (one for each dog) and 3902 columns (one for each SNP).
The name of each column should encode the chromosome number and the position as 1:5123496 for chromosome 1 and position 5123496 (from the first and second columns).

You will begin by iterating through each of the snp_lines list created above. 
You will process all genotype columns (starting from the 10th column) and convert the "x/y" genotype to x+y (total number of ALT).

Genotypes containing a '.' should be converted into a ``np.nan``. These values will be ignored later.

Input - the snp_lines parsed at the beginning of the notebook

Output - You should return the 53 x 3902 dataframe

In [43]:
snp_data=module.create_data(snp_lines)

In [48]:
# you can visualize the first 5 rows of a pandas dataframe as follows:
snp_data

,1:5123496,1:9239488,1:10894485,1:11798310,1:13061546,1:13469023,1:14254129,1:15732153,1:17432669,1:17794285,...,39:113786979,39:115191272,39:115607474,39:115954242,39:116294011,39:118625882,39:121038139,39:122458103,39:122478098,39:123278777
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,NaN,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,2.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
5,2.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,2.0,2.0,1.0,NaN,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,NaN,2.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0


## Graded Function 2 : create_target() (10 marks)

Purpose - To parse the header line and create a list of phenotype labels for the 53 dogs. We will represent yellow fur by 1 and dark fur by 0.

You should parse the header line (tab-delimited) and from the 10th element, assign 1 if yellow appears in name and 0 if dark appears in name.

Input - the header_line parsed at the beginning of the notebook

Output - a list of values(either 0 or 1).

In [35]:
target=module.create_target(header_line)
print(target[:10])

[0, 0, 0, 1, 1, 0, 0, 0, 0, 0]


## Expected Output - 

[0, 0, 0, 1, 1, 0, 0, 0, 0, 0]

We will now add the target to the snp_data dataframe under the name "yellow"

In [59]:
snp_data["target"]=target
snp_data["target"]

0

## Graded Function 3 : logistic_reg_per_snp(df)  (10 marks)

Purpose - To iterate through all SNPs and fit a logistic regression model for each SNP. 

We will use the Logit class of statsmodels.api to create the logistic regression models.
The advantage of statsmodels' version of Logistic Regression is that it returns a p-value (unlike the scikit-learn version).

The input for the model will be the dataframe column for a particular SNP and the target variable will be the target column with fur color information. 

<!-- You should not include values in the input column which are less than 0. -->

You should add a constant column to the input column. You can do that using sm.add_constant() function. 
This will guarantee that a constant $\beta_0$
Please see documentation for more information:

(https://www.statsmodels.org/stable/api.html)

(https://tedboy.github.io/statsmodels_doc/generated/generated/statsmodels.api.Logit.html)

##### Note - Please use the method='bfgs' and disp=False in the fit method of the Logit object you will create.

##### Note - Please round off the p-value to 9 digits after decimal and betavalue to 5 places after decimal

The output should be list of 3902 pvalues and a list of 3902 beta values.

In [182]:
p_values,betavalues=module.logistic_reg_per_snp(snp_data)

In [154]:
print(p_values[:10])

print(betavalues[:10])

[0.471163991, 0.250546824, 0.341485853, 0.314182679, 0.255696421, 0.120055343, 0.965506994, 0.864100297, 0.864100297, 0.143809377]
[-0.51109, -0.77511, -0.40365, 0.39437, 0.52553, -1.60262, 0.01903, 0.12674, 0.12674, 0.96828]


## Expected Output - 

[0.471163991, 0.250546824, 0.341485853, 0.314182679, 0.255696421, 0.120055343, 0.965506994, 0.864100297, 0.864100297, 0.143809377]

[-0.51109, -0.77511, -0.40365, 0.39437, 0.52553, -1.60262, 0.01903, 0.12674, 0.12674, 0.96828]

In [92]:
# You may also want to look at some crosstabs between a SNP and the target phenotype
# For example, let's take the beta with the largest absolute

betamax = np.argmax(np.abs(betavalues))
pd.crosstab(snp_data[snp_data.columns[betamax]],snp_data['target'])

target,0,1
22:55322251,,
0.0,29,14
1.0,0,6
2.0,0,3


We see that for SNP 22:55322251, every single dog in our dataset that has one or two ALT variants, has yellow fur.

## Graded Function 4 : get_top_snps() (10 marks)

Purpose - To identify the 5 SNPs whose association with dog color has the highest stastical significance. 

Statistical significance is determined in terms of p-values computed above.
The lower the p-value, the more significant the association is.

Output - list of 5 tuples, each with chromosome and position

#### Example Output Format - [('1','10'),('2','20'),('3','30'),('4','40'),('5','50')]

In [183]:
module.get_top_snps(snp_data,p_values)

[('5', '63694334'),
 ('5', '63694112'),
 ('5', '64794450'),
 ('20', '22256198'),
 ('15', '61305976')]

[9.608e-06,
 1.2193e-05,
 4.6569e-05,
 0.000123705,
 0.000139036,
 0.000139198,
 0.000144907,
 0.000147553,
 0.000157276,
 0.000159197,
 0.00019192,
 0.000216719,
 0.000293992,
 0.000308619,
 0.000401543,
 0.000402776,
 0.000437729,
 0.000449774,
 0.000480311,
 0.000510539,
 0.000541457,
 0.000546878,
 0.000590127,
 0.000703404,
 0.000704915,
 0.000801397,
 0.000809412,
 0.000841719,
 0.000843139,
 0.001005953,
 0.001042509,
 0.001054357,
 0.001068337,
 0.00115896,
 0.001172823,
 0.001179388,
 0.00119287,
 0.001193749,
 0.001225056,
 0.001368216,
 0.001378252,
 0.001407363,
 0.001606519,
 0.001617462,
 0.001617708,
 0.001617713,
 0.001617811,
 0.00162902,
 0.001795091,
 0.001816646,
 0.00188367,
 0.001925247,
 0.001969973,
 0.00198542,
 0.00217235,
 0.002250863,
 0.002276153,
 0.00236896,
 0.002403067,
 0.002421577,
 0.002671794,
 0.002714961,
 0.002744,
 0.00274435,
 0.002870337,
 0.002870352,
 0.002981217,
 0.003046569,
 0.003052359,
 0.003062265,
 0.003105799,
 0.003175022,
 0.00327

## Manhattan plot

A Manhattan plot shows -log_10(p-value) for each of the SNPs, organized by chromosome.
The Manhattan plot is commonly used to display the result of a GWAS: https://en.wikipedia.org/wiki/Manhattan_plot

In [123]:
cur_chr = '1'
cur_x = []
cur_y = []
prev_start = 0
xpos = []
xlabel = []

fig, ax = plt.subplots(figsize=(12,8))

for i in range(len(p_values)):
    
    chrm = (list(snp_data.columns)[i]).split(':')[0]
    
    if chrm != cur_chr:
        clr = 'gold' if int(cur_chr)%2 == 0 else 'darkblue'
        ax.plot(cur_x,cur_y, marker='o',linestyle='',color=clr)
        xpos.append(int((prev_start+i)/2))
        xlabel.append(cur_chr)
        cur_chr = chrm
        cur_x = []
        cur_y = []
        prev_start = i

    cur_x.append(i)
    cur_y.append(-np.log10(p_values[i]))

clr = 'gold' if int(cur_chr)%2 == 0 else 'darkblue'
ax.plot(cur_x,cur_y, marker='o',linestyle='',color=clr)
xpos.append(int((prev_start+i)/2))
xlabel.append(cur_chr)
plt.xticks(xpos, xlabel)
plt.xlabel('chromosome number')
plt.ylabel('- log10(p)')
plt.show()    

NameError: name 'plt' is not defined

Notice that there are multiple SNPs in chromosome 5 and they are close to each other.
In fact, these SNPs are on the MC1R gene (https://www.ncbi.nlm.nih.gov/gene/489652), which plays a role in skin/fur pigmentation.